# Fazer um plot com o redshift e a idade de universo como eixos usando astropy.cosmology

## Autores
Neil Crighton, Stephanie T. Douglas

## Tradução
Vinicius Cordeiro

## Objetivos
* Plotar relações usando 'matplotlib'
* Adicionar um segundo eixo em um plot do 'matplotlib'
* Relacionar distância, redshift e idade para dois diferentes tipos de cosmologia usando 'astropy.cosmology'

## Palavras-Chave
unidades, física, cosmologia, matplotlib

## Sumário

Cada redshift corresponde a uma idade do universo, então quando se plota uma quantidade em função do redshift, o gráfico resultante pode ser usado também para indicar a idade do universo. A relação entre os dois depende do tipo de cosmologia que se está assumindo, e é nesse ponto que o 'astropy.cosmology' pode ser utilizado. Nesse tutorial nós vamos mostrar como usar as ferramentas no 'astropy.cosmology' para criar um plot desse tipo:

In [ ]:
# Inicializar o matplotlib

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from IPython.display import Image
Image(filename="ang_dist.png", width=500)

Nós começamos com um objeto cosmologia (variável cosmo). Vamos criar uma cosmologia plana (o que significa que a densidade de curvatura $\Omega_k=0$) com um parâmetro de Hubble igual a $70$ km/s/Mpc e uma densidade de matéria de $\Omega_M=0.3$ para o redshift 0. A partir disso, a cosmologia `FlatLambdaCDM` calcula automaticamente que o valor da densidade de energia escura precisa ser $\Omega_\Lambda=0.7$, pois $\Omega_M + \Omega_\Lambda + \Omega_k = 1$.

In [ ]:
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

# Nesse caso nós só precisamos definir a densidade de matéria
# e o parâmetro de hubble para z=0 (a função FlatLambdaCDM já
# considera que omega_k=0)

# Note que a a unidade padrão para o parâmetro de Hubble é
# km/s/Mpc. Mesmo assim, vamos passar um objeto 'Quantidade' 
# com as unidades especificadas

cosmo = FlatLambdaCDM(H0=70*u.km/u.s/u.Mpc, Om0=0.3)

Note que nós poderíamos ter usado também uma das outras cosmologias inclusas no astropy.cosmology, como a 'WMAP9' ou a 'Planck13'.

Agora, precisamos de uma certa quantidade para plotar em função do redshift. Vamos usar a distância do diâmetro angular, que é a distância física transversal (o tamanho de uma galáxia, por exemplo) correpondente a uma certa separação angular do céu. Para calcular a distância do diâmetro angular para um intervalo de redshifts:

In [ ]:
import numpy as np
zvals = np.arange(0, 6, 0.1)
dist = cosmo.angular_diameter_distance(zvals)

Note que nós passamos um array de redshifts para 'cosmo.angular_diameter_distances', e isso produziu um array de valores de distância, um para cada redshift. Vamos então plotá-los:

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(zvals, dist)

Para checar a unidades da distância do diâmetro angular, olhamos o seu atributo unit:

In [ ]:
dist.unit

Vamos agora criar um array com algumas idades que irão aparecer no eixo superior do gráfico. Escolhemos uma série de valores de idade, correspondendo aos lugares onde queremos colocar os ticks. Talvez seja necessário que você ajuste os valores abaixo a depender do seu range de redshifts para conseguir ticks espaçados igualmente.

In [ ]:
ages = np.array([13, 10, 8, 6, 5, 4, 3, 2, 1.5, 1.2, 1])*u.Gyr

Para linkar o eixo dos redshifts com o das idades, temos que encontrar o redshift correspondente a cada uma das idades do nosso array, e a função 'z_at_value' faz exatamente isso.

In [ ]:
from astropy.cosmology import z_at_value
ageticks = [z_at_value(cosmo.age, age) for age in ages]

Agora fazemos o segundo eixo, e definimos as posições dos seus ticks (usando '.set_xticks') usando o array ageticks.

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(zvals, dist)
ax2 = ax.twiny()
ax2.set_xticks(ageticks)

Agora já temos os ticks no eixo superior nas posições corretas, mas os valores de suas labels são os redshifts, e não as idades. Para corrigir isso, podemos definir suas labels manualmente.

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(zvals, dist)
ax2 = ax.twiny()
ax2.set_xticks(ageticks)
ax2.set_xticklabels(['{:g}'.format(age) for age in ages.value])

Precisamos então garantir que os dois eixos tenham os mesmos limites de redshift. No plot acima eles podem não estar devidamente alinhados, a depender do seu setup (por exemplo, a idade do universo deve ser ~13 Gyr para z=0).

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(zvals, dist)
ax2 = ax.twiny()
ax2.set_xticks(ageticks)
ax2.set_xticklabels(['{:g}'.format(age) for age in ages.value])
zmin, zmax = 0.0, 5.9
ax.set_xlim(zmin, zmax)
ax2.set_xlim(zmin, zmax)

Quase lá. Só precisamos agora adicionar os títulos dos eixos e os ticks menores. Vamos ajustar também os limites do eixo y para evitar que suas labels fiquem muito próximas do topo do plot.

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(zvals, dist)
ax2 = ax.twiny()
ax2.set_xticks(ageticks)
ax2.set_xticklabels(['{:g}'.format(age) for age in ages.value])
zmin, zmax = 0, 5.9
ax.set_xlim(zmin, zmax)
ax2.set_xlim(zmin, zmax)
ax2.set_xlabel('Time since Big Bang (Gyr)')
ax.set_xlabel('Redshift')
ax.set_ylabel('Angular diameter distance (Mpc)')
ax.set_ylim(0, 1890)
ax.minorticks_on()

Para comparação, vamos adicionar a distância do diâmetro angular de uma cosmologia diferente, derivada dos resultados de 2013 do Planck. E finalmente, salvamos a figura em um arquivo png.

In [ ]:
from astropy.cosmology import Planck13
dist2 = Planck13.angular_diameter_distance(zvals)

fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(zvals, dist2, label='Planck 2013')
ax.plot(zvals, dist, label=
        '$h=0.7,\ \Omega_M=0.3,\ \Omega_\Lambda=0.7$')
ax.legend(frameon=0, loc='lower right')
ax2 = ax.twiny()
ax2.set_xticks(ageticks)
ax2.set_xticklabels(['{:g}'.format(age) for age in ages.value])
zmin, zmax = 0.0, 5.9
ax.set_xlim(zmin, zmax)
ax2.set_xlim(zmin, zmax)
ax2.set_xlabel('Time since Big Bang (Gyr)')
ax.set_xlabel('Redshift')
ax.set_ylabel('Angular diameter distance (Mpc)')
ax.minorticks_on()
ax.set_ylim(0, 1890)
fig.savefig('ang_dist.png', dpi=200, bbox_inches='tight')

`bbox_inches='tight'` remove automaticamente qualquer espaço em branco ao redor das margens do plot.

E terminamos!

## Exercício

Bem, quase terminamos. Note que nós calculamos os tempos no eixo superior usando a cosmologia original, e não a nova cosmologia baseada nos resultados de 2013 do Planck. Por isso, tecnicamente o eixo superior só pode ser utilizado pela cosmologia original, apesar de a diferença entre as duas ser pequena. Como exercício, você pode tentar plotar dois eixos superiores diferentes (com uma pequena separação entre os dois), para demonstrar os tempos correspondentes a cada cosmologia. Dê uma olhada na primeira resposta [dessa pergunta no Stack Overflow](http://stackoverflow.com/questions/7733693/matplotlib-overlay-plots-with-different-scales) que tem algumas dicas de como fazer isso.